In [1]:
# Installation of the libraries which we'll use: HuggingFace Transformers, Datasets and Evaluate
!pip install -q transformers datasets evaluate
!pip install imbalanced-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.8 MB/s eta 0:00:00


In [2]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.model_selection import train_test_split

from tqdm import trange
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import numpy as np

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import warnings
warnings.filterwarnings("ignore")
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

# Set the seed for generation of randomness inside the model
seed = 42
torch.manual_seed(seed)

Mounted at /content/drive


In [3]:
# Tokenize each set split
def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 32,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )

def preprocessing_batch(data_set):
    token_id = []
    attention_masks = []
    for sample in data_set:
      encoding_dict = preprocessing(sample, tokenizer)
      token_id.append(encoding_dict['input_ids'])
      attention_masks.append(encoding_dict['attention_mask'])
    token_id = torch.cat(token_id, dim = 0)
    attention_masks = torch.cat(attention_masks, dim = 0)
    return token_id,attention_masks

In [4]:
# Load the metrics
import evaluate

accuracy_metric = evaluate.load('accuracy')
precision_metric = evaluate.load('precision')
recall_metric = evaluate.load('recall')
f1_metric = evaluate.load('f1')
mcc_metric = evaluate.load('matthews_correlation')

# Compute of metrics
def compute_metrics(logits, label_ids):
  preds = np.argmax(logits, axis=1)
  accuracy = accuracy_metric.compute(predictions=preds, references=label_ids)['accuracy']
  precision = precision_metric.compute(predictions=preds, references=label_ids, average='macro')['precision']
  recall = recall_metric.compute(predictions=preds, references=label_ids, average='macro')['recall']
  f1 = f1_metric.compute(predictions=preds, references=label_ids, average='macro')['f1']
  mcc = mcc_metric.compute(predictions=preds, references=label_ids)['matthews_correlation']

  return accuracy, precision, recall, f1, mcc

In [5]:
from itertools import product

# Define hyperparameters possible values
test_and_val_ratios = [0.2, 0.3]
learning_rates = [5e-5, 3e-5]
batch_sizes = [16, 32]
epochs = [2, 3, 4, 8]
dropouts = [0.1, 0.2, 0.3]

best_test_and_val_ratio=0
best_learning_rate=0
best_batch_size=0
best_epoch=0
best_weighted_average=0

best_val_accuracy, best_val_precision, best_val_recall, best_val_f1, best_val_mcc = 0,0,0,0,0
best_test_accuracy, best_test_precision, best_test_recall, best_test_f1, best_test_mcc = 0,0,0,0,0

# Creation of a list of possible combinations
combinations = list(product(test_and_val_ratios, learning_rates, batch_sizes, epochs, dropouts))

# Training for each possible combination
for tvr, lr, bs, ep, dropout in combinations:
  TEST_AND_VAL_RATIO_VALUE = tvr
  TEST_RATIO = 0.5
  BATCH_SIZE_VALUE = bs
  LEARNING_RATE_VALUE = lr
  EPOCHS_VALUE = ep
  DROPOUT = dropout

  print("VALUES ---> tvr:" +  str(TEST_AND_VAL_RATIO_VALUE) + " bs: " + str(BATCH_SIZE_VALUE) + " lr: " + str(LEARNING_RATE_VALUE) + " ep: " + str(EPOCHS_VALUE) + " dropout: " + str(DROPOUT))

  root = "https://raw.githubusercontent.com/alfcan/CADOCS_NLU_Model/dev/dataset/augmented_dataset.csv"

  df = pd.read_csv(root, sep = ';')
  label_mapping = {'get_smells': 0, 'get_smells_date': 1, 'report': 2, 'info': 3}
  df['intent'] = df['intent'].map(label_mapping)

  dfParaphrased = pd.read_csv("https://raw.githubusercontent.com/alfcan/CADOCS_NLU_Model/dev/dataset/paraphrased_dataset.csv", sep = ';', names=["original", "paraphrase", "intent"])

  request = df.request.values
  intent = df.intent.values

  # Divide the dataset into train and test sets
  # Test set is divided into a test set and a validation set
  test_and_val_ratio = TEST_AND_VAL_RATIO_VALUE
  test_ratio = TEST_RATIO


  # Indices of the train set and temp set (validation + test sets) splits stratified by labels
  train_request, temp_request, train_intent, temp_intent = train_test_split(
      request,
      intent,
      random_state=42,
      test_size = test_and_val_ratio,
      shuffle = True,
      stratify = intent)

  # We divide then temp_idx in test_idx (test set) and val_idx ( validation set)
  test_request, val_request, test_intent, val_intent = train_test_split(
      temp_request,
      temp_intent,
      random_state=42,
      test_size = test_ratio,
      shuffle = True,
      stratify = None)

  # define undersample strategy
  undersample = RandomUnderSampler(random_state=42)

  shape_train_request = train_request.shape
  shape_train_intent = train_intent.shape

  x_train = train_request.reshape(-1, 1)
  y_train = train_intent.reshape(-1, 1)

  # fit and apply the transform
  train_request, train_intent = undersample.fit_resample(x_train, y_train)

  # Tokenizer of RoBERTa
  tokenizer = RobertaTokenizer.from_pretrained(
      'roberta-base',
      do_lower_case = True
  )

  train_request=train_request.flatten()
  train_intent=train_intent.flatten()

  train_token_id,train_attention_masks = preprocessing_batch(train_request)
  test_token_id,test_attention_masks = preprocessing_batch(test_request)
  val_token_id,val_attention_masks = preprocessing_batch(val_request)

  train_intent = torch.tensor(train_intent)
  test_intent = torch.tensor(test_intent)
  val_intent = torch.tensor(val_intent)

  batch_size = BATCH_SIZE_VALUE

  # Train, validation and test sets
  train_set = TensorDataset(train_token_id,
                          train_attention_masks,
                          train_intent)

  val_set = TensorDataset(val_token_id,
                          val_attention_masks,
                          val_intent)

  test_set = TensorDataset(test_token_id,
                          test_attention_masks,
                          test_intent)

  # Prepare DataLoader
  train_dataloader = DataLoader(
              train_set,
              sampler = RandomSampler(train_set),
              batch_size = batch_size
          )

  validation_dataloader = DataLoader(
              val_set,
              sampler = SequentialSampler(val_set),
              batch_size = batch_size
          )


  test_dataloader = DataLoader(
              test_set,
              sampler = SequentialSampler(test_set),
              batch_size = batch_size
          )

  # Load the RobertaForSequenceClassification model
  model = RobertaForSequenceClassification.from_pretrained(
      'roberta-base',
      num_labels = 4,
      output_attentions = False,
      output_hidden_states = False,
      hidden_dropout_prob = DROPOUT,
  )

  optimizer = torch.optim.AdamW(model.parameters(),
                              lr = LEARNING_RATE_VALUE,
                              eps = 1e-10
                              )

  model.cuda()

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  epochs = EPOCHS_VALUE

  for _ in trange(epochs, desc = 'Epoch'):

      # ========== Training ==========

      # Set model to training mode
      model.train()

      # Tracking variables
      tr_loss = 0
      nb_tr_examples, nb_tr_steps = 0, 0

      for step, batch in enumerate(train_dataloader):
          batch = tuple(t.to(device) for t in batch)
          b_input_ids, b_input_mask, b_labels = batch
          #reset gradient value for the new epoch
          optimizer.zero_grad()
          # Forward pass
          train_output = model(b_input_ids,
                              token_type_ids = None,
                              attention_mask = b_input_mask,
                              labels = b_labels)
          # Backward pass
          train_output.loss.backward()
          optimizer.step()
          # Update tracking variables
          tr_loss += train_output.loss.item()
          nb_tr_examples += b_input_ids.size(0)
          nb_tr_steps += 1

      # ========== Validation ==========

      # Set model to evaluation mode
      model.eval()

      for batch in validation_dataloader:
          batch = tuple(t.to(device) for t in batch)
          b_input_ids, b_input_mask, b_labels = batch
          with torch.no_grad():
            # Forward pass
            eval_output = model(b_input_ids,
                                token_type_ids = None,
                                attention_mask = b_input_mask)
            logits = eval_output.logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

      # Calculate validation metrics
      val_accuracy, val_precision, val_recall, val_f1, val_mcc = compute_metrics(logits, label_ids)
      '''
      print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
      print('\t - Validation Accuracy: {:.4f}'.format(val_accuracy))
      print('\t - Validation Precision: {:.4f}'.format(val_precision))
      print('\t - Validation Recall: {:.4f}'.format(val_recall))
      print('\t - Validation F1: {:.4f}'.format(val_f1))
      print('\t - Validation MCC: {:.4f}\n'.format(val_mcc))
      '''

  for batch in test_dataloader:
      batch = tuple(t.to(device) for t in batch)
      b_input_ids, b_input_mask, b_labels = batch

      eval_output = model(b_input_ids,
                          token_type_ids = None,
                          attention_mask = b_input_mask)
      logits = eval_output.logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()

  # Calculate test metrics
  test_accuracy, test_precision, test_recall, test_f1, test_mcc = compute_metrics(logits, label_ids)
  '''
  print('\t - Test Accuracy: {:.4f}'.format(test_accuracy))
  print('\t - Test Precision: {:.4f}'.format(test_precision))
  print('\t - Test Recall: {:.4f}'.format(test_recall))
  print('\t - Test F1: {:.4f}'.format(test_f1))
  print('\t - Test MCC: {:.4f}\n'.format(test_mcc))
  '''
  correct_get_smells, correct_get_smells_date, correct_report, correct_info=0, 0, 0, 0

  total_get_smells, total_get_smells_date, total_report, total_info=0, 0, 0, 0

  for index, row in dfParaphrased.iterrows():
    request_original=row["original"]
    request=row["paraphrase"]
    intent=row["intent"]

    request_ids = tokenizer.encode_plus(
                            request,
                            add_special_tokens = True,
                            max_length = 32,
                            pad_to_max_length = True,
                            return_attention_mask = True,
                            return_tensors = 'pt'
                      )

    input_ids = torch.clone(request_ids['input_ids']).detach().to(model.device)

    with torch.no_grad():
        output = model(input_ids)

    predictions = torch.softmax(output.logits, dim=1)
    confidence = predictions.max(dim=1).values.detach().cpu().numpy()
    predicted_class = predictions.argmax(dim=1).flatten().cpu().numpy()



    request_original_ids = tokenizer.encode_plus(
                            request_original,
                            add_special_tokens = True,
                            max_length = 32,
                            pad_to_max_length = True,
                            return_attention_mask = True,
                            return_tensors = 'pt'
                      )

    input_original_ids = torch.clone(request_original_ids['input_ids']).detach().to(model.device)

    with torch.no_grad():
        output_original = model(input_original_ids)

    predictions_original = torch.softmax(output_original.logits, dim=1)
    confidence_original = predictions_original.max(dim=1).values.detach().cpu().numpy()
    predicted_original_class = predictions_original.argmax(dim=1).flatten().cpu().numpy()

    for label, index in label_mapping.items():
        if index == predicted_class:
            predicted_label = label
            break
    for label, index in label_mapping.items():
        if index == predicted_original_class:
            predicted_original_label = label
            break

    if intent == "get_smells":
      total_get_smells=total_get_smells+1
      if intent == predicted_label == predicted_original_label and confidence_original-confidence < 0.2:
          correct_get_smells=correct_get_smells+1
    elif intent == "get_smells_date":
      total_get_smells_date=total_get_smells_date+1
      if intent == predicted_label == predicted_original_label and confidence_original-confidence < 0.2:
          correct_get_smells_date=correct_get_smells_date+1
    elif intent == "report":
      total_report=total_report+1
      if intent == predicted_label == predicted_original_label and confidence_original-confidence < 0.2:
          correct_report=correct_report+1
    elif intent == "info":
      total_info=total_info+1
      if intent == predicted_label == predicted_original_label and confidence_original-confidence < 0.2:
          correct_info=correct_info+1


    # print("Correct class: "+intent+" --- Predicted class: " + predicted_label + " --- " + request+"\n Confidence: "+str(confidence)+" --- Difference between original confidence and paraphrased confidence: "+str(confidence_original-confidence)+"\n")


  percentage_get_smells=correct_get_smells/total_get_smells
  percentage_get_smells_date=correct_get_smells_date/total_get_smells_date
  percentage_report=correct_report/total_report
  percentage_info=correct_info/total_info
  '''
  print("Percentage get_smells: "+ str(percentage_get_smells))
  print("Percentage get_smells_date: "+ str(percentage_get_smells_date))
  print("Percentage report: "+ str(percentage_report))
  print("Percentage info: "+ str(percentage_info))
  '''
  sum_of_requests= total_get_smells + total_get_smells_date + total_report + total_info
  weighted_average=((percentage_get_smells*total_get_smells)+(percentage_get_smells_date*total_get_smells_date)+(percentage_report*total_report)+(percentage_info*total_info))/(sum_of_requests)
  '''
  print("Weighted average: "+str(weighted_average))
  '''
  if weighted_average>best_weighted_average:
    best_weighted_average=weighted_average
    best_test_and_val_ratio=tvr
    best_learning_rate=lr
    best_batch_size=bs
    best_epoch=ep
    best_val_accuracy, best_val_precision, best_val_recall, best_val_f1, best_val_mcc=val_accuracy, val_precision, val_recall, val_f1, val_mcc
    best_test_accuracy, best_test_precision, best_test_recall, best_test_f1, best_test_mcc = test_accuracy, test_precision, test_recall, test_f1, test_mcc
    best_model=model

    print("New best combination\n Weighted average: "+str(weighted_average)+", test_and_val_ratio: "+str(tvr)+", learning_rate: "+str(lr)+", batch_size: "+str(bs)+", epoch: "+str(ep) + ", dropout: " + str(dropout) + "\n")

    print('\t - Validation Accuracy: {:.4f}'.format(best_val_accuracy))
    print('\t - Validation Precision: {:.4f}'.format(best_val_precision))
    print('\t - Validation Recall: {:.4f}'.format(best_val_recall))
    print('\t - Validation F1: {:.4f}'.format(best_val_f1))
    print('\t - Validation MCC: {:.4f}\n'.format(best_val_mcc)+"\n")

    print('\t - Test Accuracy: {:.4f}'.format(best_test_accuracy))
    print('\t - Test Precision: {:.4f}'.format(best_test_precision))
    print('\t - Test Recall: {:.4f}'.format(best_test_recall))
    print('\t - Test F1: {:.4f}'.format(best_test_f1))
    print('\t - Test MCC: {:.4f}\n'.format(best_test_mcc)+"\n\n\n")


print("\n\n\n------------------------------------------\nBest combination\n Weighted average: "+str(best_weighted_average)+", test_and_val_ratio: "+str(best_test_and_val_ratio)+", learning_rate: "+str(best_learning_rate)+", batch_size: "+str(best_batch_size)+", epoch: "+str(best_epoch) + ", dropout: " + str(dropout) + "\n")

print('\t - Validation Accuracy: {:.4f}'.format(best_val_accuracy))
print('\t - Validation Precision: {:.4f}'.format(best_val_precision))
print('\t - Validation Recall: {:.4f}'.format(best_val_recall))
print('\t - Validation F1: {:.4f}'.format(best_val_f1))
print('\t - Validation MCC: {:.4f}\n'.format(best_val_mcc)+"\n")

print('\t - Test Accuracy: {:.4f}'.format(best_test_accuracy))
print('\t - Test Precision: {:.4f}'.format(best_test_precision))
print('\t - Test Recall: {:.4f}'.format(best_test_recall))
print('\t - Test F1: {:.4f}'.format(best_test_f1))
print('\t - Test MCC: {:.4f}\n'.format(best_test_mcc)+"\n\n\n")

torch.save(best_model, "./CADOCS_eng_model.pth")

VALUES ---> tvr:0.2 bs: 16 lr: 5e-05 ep: 2 dropout: 0.1


Epoch: 100%|██████████| 2/2 [00:09<00:00,  4.59s/it]


New best combination
 Weighted average: 0.6857142857142857, test_and_val_ratio: 0.2, learning_rate: 5e-05, batch_size: 16, epoch: 2, dropout: 0.1

	 - Validation Accuracy: 1.0000
	 - Validation Precision: 1.0000
	 - Validation Recall: 1.0000
	 - Validation F1: 1.0000
	 - Validation MCC: 1.0000


	 - Test Accuracy: 1.0000
	 - Test Precision: 1.0000
	 - Test Recall: 1.0000
	 - Test F1: 1.0000
	 - Test MCC: 1.0000




VALUES ---> tvr:0.2 bs: 16 lr: 5e-05 ep: 2 dropout: 0.2


Epoch: 100%|██████████| 2/2 [00:06<00:00,  3.23s/it]


VALUES ---> tvr:0.2 bs: 16 lr: 5e-05 ep: 2 dropout: 0.3


...

VALUES ---> tvr:0.3 bs: 32 lr: 3e-05 ep: 8 dropout: 0.2


Epoch: 100%|██████████| 8/8 [00:20<00:00,  2.51s/it]


VALUES ---> tvr:0.3 bs: 32 lr: 3e-05 ep: 8 dropout: 0.3


Epoch: 100%|██████████| 8/8 [00:19<00:00,  2.48s/it]





------------------------------------------
Best combination
 Weighted average: 0.9714285714285714, test_and_val_ratio: 0.2, learning_rate: 5e-05, batch_size: 16, epoch: 8, dropout: 0.3

	 - Validation Accuracy: 1.0000
	 - Validation Precision: 1.0000
	 - Validation Recall: 1.0000
	 - Validation F1: 1.0000
	 - Validation MCC: 1.0000


	 - Test Accuracy: 1.0000
	 - Test Precision: 1.0000
	 - Test Recall: 1.0000
	 - Test F1: 1.0000
	 - Test MCC: 1.0000




